1. [Classic 3](#Classic_3)
- [NG5](#NG5)
- [NG20](#NG20)
- [Reuters8](#Reuteurs_8)
    


## Dépendances

In [ ]:
import numpy as np#utils
import os     

from gensim.models import Word2Vec#gensim
from gensim.models import FastText
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer#sci-kit learn
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.decomposition import TruncatedSVD


# Classic_3
## Data
On se place au même niveau que le dossier classic qui contient toutes les données

In [ ]:
all_files = os.listdir("classic/")   
print(all_files)  

In [ ]:
for i in range(len(all_files)):
    all_files[i]="classic/"+all_files[i]

In [ ]:
all_files[47]

In [ ]:

text_labels=[]
start=len("classic/")
for path in all_files:
    text_labels.append(path[start:start+4])

In [ ]:
text_labels[0:8
           ]

In [ ]:
label_dict={'cisi':0,'cran':1,'med.':2}

Le document n°47 (med.000423) est vide => on vérifie la taille du document avant d'ajouter le label, et, ainsi on ne tient plus compte de ce document.

In [ ]:

data=[]
for file_path in all_files:
    with open(file_path) as file:
        doc=[]
        for line in file:
            doc+=simple_preprocess(remove_stopwords(line))
        data.append(doc)

In [ ]:
print(data[0])

In [ ]:
joined_data=[]
for doc in data:
    joined_data.append(" ".join(doc))

In [ ]:
len(all_files),len(data),len(joined_data)

In [ ]:
labels=[]
for i,label in enumerate(text_labels):
    if len(data[i])!=0:
        labels.append(label_dict[label])

In [ ]:
len(labels)

In [ ]:
labels[0:8]

## Term Document Frequency Matrix


Commenter tous les paramètres

In [ ]:
tf=TfidfVectorizer(input='content',lowercase=False, use_idf=True,min_df=1, sublinear_tf=False)
matrix=tf.fit_transform(joined_data)
vocabulary = tf.vocabulary_

In [ ]:
matrix.shape

In [ ]:
np.array(matrix[0].todense())

In [ ]:
mot='developing'
index_mot=vocabulary[mot]
index_doc=0
np.array(matrix[index_doc].todense())[0][index_mot]

In [ ]:
print(vocabulary)

## Embedding
### Word2Vec

In [ ]:
model = Word2Vec(
        data,
        size=150,#taile du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.train(data, total_examples=len(data), epochs=10)


In [ ]:
model.wv.most_similar(positive="disorders",topn=6)

### FastText


In [ ]:
model = FastText(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.build_vocab(data,update=True)
model.train(data, total_examples=len(data), epochs=10)

### Sans tenir compte de TF-IDF

In [ ]:
D=[]
for document in data:
    D_doc=[]
    for mot in document:
        D_doc.append(model.wv.word_vec(mot))
    D.append(D_doc)

###  En tenant compte de TF-IDF
Sinon commenter indexmot, valeurtf 

In [ ]:
D=[]
for index_doc, document in enumerate(data):
    D_doc=[]
    for mot in document:
        index_mot=vocabulary[mot]
        valeur_tf_idf=np.array(matrix[index_doc].todense())[0][index_mot]
        D_doc.append(valeur_tf_idf*model.wv.word_vec(mot))
    D.append(D_doc)

In [ ]:
len(D),len(D[0]),len(D[0][0])

In [ ]:
for idx, document in enumerate(D):
    if len(document)==0:
        print("doc vide", idx)

In [ ]:
D[47],data[47]

## SVD


Le document n°47 (med.000423) est vide => on vérifie la taille du document avant de faire la transformation SVD afin de ne pas avoir d'erreur, et, ainsi on ne tient plus compte de ce document (i.e. on ne l'ajoute pas dans la matrice M).

In [ ]:
M2=[]### document représenté par u1 
M3=[]### document représenté par u1 + u2 
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)

for document in D:
    if len(document)!=0:
        svd.fit(document)

        M2.append(svd.components_[0])#[0] to transform 2D array to 1D

        M3.append(svd.components_[0]+svd.components_[1])#u1 + u2

In [ ]:
len(M2)

On a 3890 doc au lieu de 3891 à cause du document vide 

## Clustering
### K-Means

In [ ]:
M3[1]

In [ ]:
kmeans = KMeans(n_clusters=3,
               init='k-means++',
               n_init=10,
               n_jobs=-1).fit(M3)



In [ ]:
y_kmeans = kmeans.predict(M3)

In [ ]:
y_kmeans[0:10]

In [ ]:
y_kmeans.shape

In [ ]:
print("%.3f" % adjusted_rand_score(labels, y_kmeans), "%.3f" % accuracy_score(labels, y_kmeans), "%.3f" % normalized_mutual_info_score(labels, y_kmeans))

Avant le preproccess on avait  
(0.731, 0.234, 0.659) pour u1

et  
(0.676, 0.0589, 0.591) pour u1+u2


# NG5
## Data

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = [
'rec.motorcycles',
'rec.sport.baseball',
'comp.graphics',
'sci.space',
'talk.politics.mideast'
]
ng5 = fetch_20newsgroups(categories=categories)
documents = ng5.data

In [ ]:
labels=ng5.target

### Preprocessing

In [ ]:
data=[]
for doc in documents:
    data.append(simple_preprocess(remove_stopwords(doc)))
print(len(data),"\n",data[0])

In [ ]:
labels[0],ng5.target_names[labels[0]]

## Term Document Frequency Matrix

In [ ]:
joined_data=[]
for doc in data:
    joined_data.append(" ".join(doc))

In [ ]:
tf=TfidfVectorizer(input='content',lowercase=False, use_idf=True,min_df=1, sublinear_tf=False)
matrix=tf.fit_transform(joined_data)
vocabulary = tf.vocabulary_

## Embedding
### Word2Vec

In [ ]:
model = Word2Vec(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.train(data, total_examples=len(data), epochs=10)


In [ ]:
model.wv.most_similar(positive="team",topn=6)

### FastText


In [ ]:
model = FastText(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.build_vocab(data,update=True)
model.train(data, total_examples=len(data), epochs=10)

### Sans tenir compte de TF-IDF

In [ ]:
D=[]
for document in data:
    D_doc=[]
    for mot in document:
        D_doc.append(model.wv.word_vec(mot))
    D.append(D_doc)

###  En tenant compte de TF-IDF


In [ ]:
D=[]
for index_doc, document in enumerate(data):
    D_doc=[]
    for mot in document:
        index_mot=vocabulary[mot]
        valeur_tf_idf=np.array(matrix[index_doc].todense())[0][index_mot]
        D_doc.append(valeur_tf_idf*model.wv.word_vec(mot))
    D.append(D_doc)

In [ ]:
len(D),len(D[0]),len(D[0][0])

In [ ]:
for idx, document in enumerate(D):
    if len(document)==0:
        print("doc vide", idx)

Pas de document vide cette fois ci !

## SVD



In [ ]:
M2=[]### document représenté par u1 
M3=[]### document représenté par u1 + u2 
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)

for document in D:
    svd.fit(document)
    
    M2.append(svd.components_[0])#[0] to transform 2D array to 1D
    
    M3.append(svd.components_[0]+svd.components_[1])#u1 + u2

In [ ]:
len(M2)

## Clustering
### K-Means

In [ ]:
M3[1]

In [ ]:
kmeans = KMeans(n_clusters=5,
               init='k-means++',
               n_init=10,
               n_jobs=-1).fit(M3)



In [ ]:
y_kmeans = kmeans.predict(M3)

In [ ]:
y_kmeans[0:10]

In [ ]:
y_kmeans.shape

In [ ]:
print("%.3f" % adjusted_rand_score(labels, y_kmeans), "%.3f" % accuracy_score(labels, y_kmeans), "%.3f" % normalized_mutual_info_score(labels, y_kmeans))

Avant le preprocessing, on avait  
(ARI, Clustering Accuracy et NMI)  
(0.0454, 0.194, 0.0620)

# NG20
## Data

In [ ]:
from sklearn.datasets import fetch_20newsgroups

ng20 = fetch_20newsgroups()
documents = ng20.data
labels=ng20.target

### Preprocessing

In [ ]:
data=[]
for doc in documents:
    data.append(simple_preprocess(remove_stopwords(doc)))
print(len(data),"\n",data[0])

In [ ]:
labels[0],ng20.target_names[labels[0]]

## Term Document Frequency Matrix

In [ ]:
joined_data=[]
for doc in data:
    joined_data.append(" ".join(doc))

In [ ]:
tf=TfidfVectorizer(input='content',lowercase=False, use_idf=True,min_df=1, sublinear_tf=False)
matrix=tf.fit_transform(joined_data)
vocabulary = tf.vocabulary_

## Embedding
### Word2Vec

In [ ]:
model = Word2Vec(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.train(data, total_examples=len(data), epochs=10)


In [ ]:
model.wv.most_similar(positive="team",topn=6)

### FastText


In [ ]:
model = FastText(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.build_vocab(data,update=True)
model.train(data, total_examples=len(data), epochs=10)

### Sans tenir compte de TF-IDF

In [ ]:
D=[]
for document in data:
    D_doc=[]
    for mot in document:
        D_doc.append(model.wv.word_vec(mot))
    D.append(D_doc)

###  En tenant compte de TF-IDF


In [ ]:
D=[]
for index_doc, document in enumerate(data):
    D_doc=[]
    for mot in document:
        index_mot=vocabulary[mot]
        valeur_tf_idf=np.array(matrix[index_doc].todense())[0][index_mot]
        D_doc.append(valeur_tf_idf*model.wv.word_vec(mot))
    D.append(D_doc)

In [ ]:
len(D),len(D[0]),len(D[0][0])

In [ ]:
for idx, document in enumerate(D):
    if len(document)==0:
        print("doc vide", idx)

Pas de document vide cette fois ci !

## SVD


In [ ]:
M2=[]### document représenté par u1 
M3=[]### document représenté par u1 + u2 
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)

for document in D:
    svd.fit(document)
    
    M2.append(svd.components_[0])#[0] to transform 2D array to 1D
    
    M3.append(svd.components_[0]+svd.components_[1])#u1 + u2

In [ ]:
len(M2)

## Clustering
### K-Means

In [ ]:
M2[1]

In [ ]:
kmeans = KMeans(n_clusters=20,
               init='k-means++',
               n_init=10,
               n_jobs=-1).fit(M3)



In [ ]:
y_kmeans = kmeans.predict(M3)

In [ ]:
y_kmeans[0:10]

In [ ]:
y_kmeans.shape

In [ ]:
print("%.3f" % adjusted_rand_score(labels, y_kmeans), "%.3f" % accuracy_score(labels, y_kmeans), "%.3f" % normalized_mutual_info_score(labels, y_kmeans))

# Reuteurs_8
## Data

In [ ]:
all_files = os.listdir("./reuters8_data")   

print(all_files)  

In [ ]:
label_file  = open( "./reuters8_data/reuters8_labels.txt" );
text_labels=[]

for line in label_file.readlines():
    
    y =  line.splitlines()
    
    text_labels.append(y[0])

len(text_labels)    


In [ ]:
text_labels[0:10]
unique_labels=set(text_labels)
unique_labels

In [ ]:
label_dict= dict(zip(unique_labels, range(0,8)))#{'cisi':0,'cran':1,'med.':2}
label_dict

In [ ]:
labels=[]
for label in text_labels:
    labels.append(label_dict[label])
len(labels)

In [ ]:
labels[0:10]

In [ ]:
tmp = []

f = open('./reuters8_data/reuters8_raw.txt', 'r')
    
tmp = f.read()
tmp=tmp.split("\n")
f.close()

In [ ]:
len(tmp)

In [ ]:
tmp[0]

Le dernier document est vide !

In [ ]:
tmp[5485]

In [ ]:
tmp.pop(5485)

### Preprocessing

In [ ]:
data=[]
for doc in tmp:
    data.append(simple_preprocess(remove_stopwords(doc)))
print(len(data),"\n",data[0])

In [ ]:
labels[0],text_labels[0]

## Term Document Frequency Matrix

In [ ]:
joined_data=[]
for doc in data:
    joined_data.append(" ".join(doc))

In [ ]:
tf=TfidfVectorizer(input='content',lowercase=False, use_idf=True,min_df=1, sublinear_tf=False)
matrix=tf.fit_transform(joined_data)
vocabulary = tf.vocabulary_

## Embedding
### Word2Vec

In [ ]:
model = Word2Vec(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.train(data, total_examples=len(data), epochs=10)


In [ ]:
model.wv.most_similar(positive="products",topn=6)

### FastText


In [ ]:
model = FastText(
        data,
        size=150,#taille du vecteur d'embedding pour chaque mot
        window=10,
        min_count=1,
        workers=10)
model.build_vocab(data,update=True)
model.train(data, total_examples=len(data), epochs=10)

### Sans tenir compte de TF-IDF

In [ ]:
D=[]
for document in data:
    D_doc=[]
    for mot in document:
        D_doc.append(model.wv.word_vec(mot))
    D.append(D_doc)

###  En tenant compte de TF-IDF


In [ ]:
D=[]
for index_doc, document in enumerate(data):
    D_doc=[]
    for mot in document:
        index_mot=vocabulary[mot]
        valeur_tf_idf=np.array(matrix[index_doc].todense())[0][index_mot]
        D_doc.append(valeur_tf_idf*model.wv.word_vec(mot))
    D.append(D_doc)

In [ ]:
len(D),len(D[0]),len(D[0][0])

In [ ]:
for idx, document in enumerate(D):
    if len(document)==0:
        print("doc vide", idx)

Pas de document vide cette fois ci !

## SVD


In [ ]:
M2=[]### document représenté par u1 
M3=[]### document représenté par u1 + u2 
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)

for document in D:
    svd.fit(document)
    
    M2.append(svd.components_[0])#[0] to transform 2D array to 1D
    
    M3.append(svd.components_[0]+svd.components_[1])#u1 + u2

In [ ]:
len(M2)

## Clustering
### K-Means

In [ ]:
M3[1]

In [ ]:
kmeans = KMeans(n_clusters=8,
               init='k-means++',
               n_init=10,
               n_jobs=-1).fit(M3)



In [ ]:
y_kmeans = kmeans.predict(M3)

In [ ]:
y_kmeans[0:10]

In [ ]:
y_kmeans.shape

In [ ]:
print("%.3f" % adjusted_rand_score(labels, y_kmeans), "%.3f" % accuracy_score(labels, y_kmeans), "%.3f" % normalized_mutual_info_score(labels, y_kmeans))

## Conclusion
### En utilisant Word2Vec

Methode (ARI, Clustering Accuracy et NMI)/Dataset | Classic | NG5 | NG20 | R8  
--- | --- | --- | --- | ---  
document représenté par u 1 sans utiliser tf-idf | (0.731, 0.234, 0.659)| (0.120, 0.229, 0.252) | (0.103, 0.048, 0.235) |  0.397 0.010 0.538 
document représenté par u 1 en utilisant tf-idf | (0.922, 0.264, 0.877) | (0.450, 0.193, 0.463) | (0.175, 0.033, 0.305) |   0.369 0.014 0.496
document représenté par u 1 + u 2 sans utiliser tf-idf | (0.672, 0.268, 0.617) | (0.144, 0.148, 0.199) | (0.062, 0.060, 0.163) |  (0.200, 0.071, 0.305)
document représenté par u 1 + u 2 en utilisant tf-idf | (0.705, 0.0576, 0.630) | (0.320, 0.219, 0.330) | (0.146, 0.0597, 0.264) |  0.188 0.051 0.331

### En utilisant fastText

Methode (ARI, Clustering Accuracy et NMI)/Dataset | Classic | NG5 | NG20 | R8  
--- | --- | --- | --- | ---  
document représenté par u 1 sans utiliser tf-idf | 0.923 0.364 0.884 | 0.039 0.199 0.069 | 0.023 0.049 0.077 | 0.358 0.137 0.469
document représenté par u 1 en utilisant tf-idf  | 0.871 0.955 0.813| 0.187 0.215 0.240 | 0.107 0.049 0.217 | 0.345 0.130 0.453
document représenté par u 1 + u 2 sans utiliser tf-idf  | 0.766 0.037 0.710 | 0.029 0.198 0.034 | 0.015 0.041 0.055 | 0.210 0.112 0.324
document représenté par u 1 + u 2 en utilisant tf-idf | 0.745 0.907 0.667 | 0.183 0.095 0.239 | 0.085 0.085 0.182 | 0.237 0.271 0.342

1. [Classic 3](#Classic_3)
- [NG5](#NG5)
- [NG20](#NG20)
- [Reuters8](#Reuteurs_8)
    
